<a href="https://colab.research.google.com/github/t-nakatani/signate_stu22/blob/main/signate_stu_v2_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ROOT = "/content/drive/MyDrive/Colab Notebooks/signate/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://qiita.com/yamaru/items/63a342c844cff056a549

①データサイエンティスト（DS）  
②機械学習エンジニア（ML Engineer）  
③ソフトウェアエンジニア（Software Engineer）  
④コンサルタント（Consultant）   

In [ ]:
jobdic = dict(zip([1, 2, 3, 4], ['DataScientist', 'ML Engineer', 'Software Engineer', 'Consultant']))
jobdic

{1: 'DataScientist', 2: 'ML Engineer', 3: 'Software Engineer', 4: 'Consultant'}

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
#学習用データと評価用データの読み込み
# train = pd.read_csv(os.path.join(ROOT, "train.csv"))
test = pd.read_csv(os.path.join(ROOT, "test.csv"))

In [ ]:
test

,id,description
0,1516,<li>Building decision-making models and propos...
1,1517,<li>Educate homeowners on the benefits of sola...
2,1518,"<li><span>Design, develop, document, and imple..."
3,1519,<li>Apply advanced technical expertise and ski...
4,1520,<li>Project manage and deliver against our roa...
...,...,...
1512,3028,<li> Manages the development of interface requ...
1513,3029,<li>Lead the implementation of new statistical...
1514,3030,<li>Configure Zuora to fulfill use cases</li><...
1515,3031,"<li>Gather, translate, and document stakeholde..."


In [ ]:
test['jobflag'] = -1
combined = train.append(test, ignore_index=True)

NameError: ignored

In [ ]:
import re
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def cleaning(texts):
    clean_texts = []
    for text in texts:
        # htmlタグを削除
        text = remove_tag(text)
        # #アルファベット以外をスペースに置き換え
        clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        # #単語長が3文字以下のものは削除する
        clean_short_tokenized = [word for word in clean_punc.split() if len(word) > 3]
        #ステミング
        # clean_normalize = [stemmer.stem(word) for word in clean_short_tokenized]
        #単語同士をスペースでつなぎ, 文章に戻す
        # clean_text = ' '.join(clean_normalize)
        clean_text = ' '.join(clean_short_tokenized)
        
        clean_texts.append(clean_text)
    return clean_texts

def remove_tag(x):
    p = re.compile(r"<[^>]*?>")
    return p.sub('',x)
    
test_cleaned = test.copy()
test_cleaned['description'] = cleaning(test_cleaned['description'])

In [ ]:
!pip install -q transformers==3

     |████████████████████████████████| 754 kB 8.1 MB/s 
     |████████████████████████████████| 1.2 MB 37.7 MB/s 
     |████████████████████████████████| 880 kB 38.0 MB/s 
     |████████████████████████████████| 3.0 MB 32.5 MB/s 


In [ ]:
import numpy as np
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from torch import optim
from torch import cuda
import time
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
# Datasetの定義
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):  # len(Dataset)で返す値を指定
    return len(self.y)

  def __getitem__(self, index):  # Dataset[index]で返す値を指定
    text = self.X[index]
    inputs = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      pad_to_max_length=True,
      truncation=True
    )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']

    return {
      'ids': torch.LongTensor(ids),
      'mask': torch.LongTensor(mask),
      'labels': torch.Tensor(self.y[index])
    }

In [ ]:
# from sklearn.model_selection import train_test_split

# y = pd.get_dummies(train_cleaned.jobflag).values
y_test = pd.get_dummies(test_cleaned.jobflag).values

# X_train, X_val, y_train, y_val = train_test_split(train['description'].values, y, test_size=0.2,random_state=109) 

# Datasetの作成
max_len = 400
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# dataset_train = CreateDataset(X_train, y_train, tokenizer, max_len)
# dataset_val = CreateDataset(X_val, y_val, tokenizer, max_len)
dataset_test = CreateDataset(test['description'], y_test, tokenizer, max_len)

# for var in dataset_train[0]:
#   print(f'{var}: {dataset_train[0][var]}')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# BERT分類モデルの定義
class BERTClass(torch.nn.Module):
  def __init__(self, drop_rate, otuput_size):
    super().__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.drop = torch.nn.Dropout(drop_rate)
    self.fc = torch.nn.Linear(768, otuput_size)  # BERTの出力に合わせて768次元を指定

  def forward(self, ids, mask):
    _, out = self.bert(ids, attention_mask=mask)
    out = self.fc(self.drop(out))
    return out

In [ ]:
def calculate_loss_and_accuracy(model, criterion, loader, device):
  """ 損失・正解率を計算"""
  model.eval()
  loss = 0.0
  total = 0
  correct = 0
  with torch.no_grad():
    for data in tqdm(loader):
      # デバイスの指定
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      labels = data['labels'].to(device)

      # 順伝播
      outputs = model(ids, mask)

      # 損失計算
      loss += criterion(outputs, labels).item()

      # 正解率計算
      pred = torch.argmax(outputs, dim=-1).cpu().numpy() # バッチサイズの長さの予測ラベル配列
      labels = torch.argmax(labels, dim=-1).cpu().numpy()  # バッチサイズの長さの正解ラベル配列
      total += len(labels)
      correct += (pred == labels).sum().item()

  return loss / len(loader), correct / total

In [ ]:
model = torch.load(os.path.join(ROOT, 'v1.pth'))

In [ ]:
# 正解率の算出
def predict(model, dataset, device):
  # Dataloaderの作成
  loader = DataLoader(dataset, batch_size=8, shuffle=False)

  model.eval()
  total = 0
  preds = []
  with torch.no_grad():
    for data in tqdm(loader):
      # デバイスの指定
      ids = data['ids'].to(device)
      mask = data['mask'].to(device)
      labels = data['labels'].to(device)

      # 順伝播 + 予測値の取得 + 正解数のカウント
      outputs = model.forward(ids, mask)
      pred = torch.argmax(outputs, dim=-1).cpu().numpy()
      preds.append(pred)

  return preds
device = 'cuda' if cuda.is_available() else 'cpu'
preds = predict(model, dataset_test, device)

100%|██████████| 190/190 [00:28<00:00,  6.57it/s]


In [ ]:
flatten = [p+1 for arr in preds for p in arr]
len(flatten)

1517

In [ ]:
df_submit = pd.DataFrame([[id, pred] for id, pred in zip(test.id, flatten)])
df_submit.to_csv(os.path.join(ROOT, 'v2_submit.csv'), index=None, header=None)

In [ ]:
df_submit

,0,1
0,1516,1
1,1517,4
2,1518,3
3,1519,4
4,1520,3
...,...,...
1512,3028,3
1513,3029,1
1514,3030,3
1515,3031,4
